In [1]:
import cv2
import numpy as np
import math
from scipy import ndimage

In [2]:
img = cv2.imread('Shaqi_jrvej.jpg', 0)

In [3]:
gaussian = ndimage.gaussian_filter(img, sigma=1.4)
cv2.imshow('Gray Image', gaussian)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
sobel_x = cv2.Sobel(gaussian, cv2.CV_64F, 1, 0, ksize=5)
sobel_y = cv2.Sobel(gaussian, cv2.CV_64F, 0, 1, ksize=5)

mag = np.sqrt(sobel_x**2 + sobel_y**2)
mag = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
theta = np.arctan2(sobel_y, sobel_x)
theta = np.round(theta / 135) * 135

cv2.imshow('Gradient Image', mag)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
magnitude_suppressed = np.zeros_like(mag)
for i in range(1, mag.shape[0]-1):
    for j in range(1, mag.shape[1]-1):
        if (angle[i][j] >= 0 and angle[i][j] < 22.5) or (angle[i][j] >= 157.5 and angle[i][j] <= 180):
            q = mag[i][j+1]
            r = mag[i][j-1]
        elif angle[i][j] >= 22.5 and angle[i][j] < 67.5:
            q = mag[i+1][j-1]
            r = mag[i-1][j+1]
        elif angle[i][j] >= 67.5 and angle[i][j] < 112.5:
            q = mag[i+1][j]
            r = mag[i-1][j]
        elif angle[i][j] >= 112.5 and angle[i][j] < 157.5:
            q = mag[i-1][j-1]
            r = mag[i+1][j+1]
        if mag[i][j] >= q and mag[i][j] >= r:
            magnitude_suppressed[i][j] = mag[i][j]
            
cv2.imshow('Non-Maximum Image', magnitude_suppressed)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
high_threshold = np.max(magnitude_suppressed)*0.0025
low_threshold = high_threshold*0.5

strong_edges = np.zeros_like(magnitude_suppressed)
weak_edges = np.zeros_like(magnitude_suppressed)
for i in range(magnitude_suppressed.shape[0]):
    for j in range(magnitude_suppressed.shape[1]):
        if magnitude_suppressed[i][j] >= high_threshold:
            strong_edges[i][j] = magnitude_suppressed[i][j]
        elif magnitude_suppressed[i][j] >= low_threshold:
            weak_edges[i][j] = magnitude_suppressed[i][j]
            
cv2.imshow('Double Threshold Image', strong_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
final_edges = np.zeros_like(strong_edges)
for i in range(1, strong_edges.shape[0]-1):
    for j in range(1, strong_edges.shape[1]-1):
        if strong_edges[i][j] != 0:
            final_edges[i][j] = strong_edges[i][j]
            if weak_edges[i-1][j-1] != 0:
                final_edges[i-1][j-1] = strong_edges[i][j]
            if weak_edges[i-1][j] != 0:
                final_edges[i-1][j] = strong_edges[i][j]
            if weak_edges[i-1][j+1] != 0:
                final_edges[i-1][j+1] = strong_edges[i][j]
            if weak_edges[i][j-1] != 0:
                final_edges[i][j-1] = strong_edges[i][j]
            if weak_edges[i][j+1] != 0:
                final_edges[i][j+1] = strong_edges[i][j]
            if weak_edges[i+1][j-1] != 0:
                final_edges[i+1][j-1] = strong_edges[i][j]
            if weak_edges[i+1][j] != 0:
                final_edges[i+1][j] = strong_edges[i][j]
            if weak_edges[i+1][j+1] != 0:
                final_edges[i+1][j+1] = strong_edges[i][j]
                
cv2.imshow('Final Edge Image', final_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import numpy as np
import os
import cv2

In [2]:
def Canny_detector(img, weak_th=None, strong_th=None):
    # conversion of image to grayscale
    img = cv2.resize(img, (400, 400), interpolation = cv2.INTER_AREA)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Grayscale', img)
    # cv2.imwrite("Grayscale.jpg", img)

    # Noise reduction step
    img = cv2.GaussianBlur(img, (5, 5), 1)
    cv2.imshow('Noise Reduction', img)
    # cv2.imwrite("Noise Reduction.jpg", img)

    # Calculating the gradients
    gx = cv2.Sobel(np.float32(img), cv2.CV_64F, 1, 0, 3)
    gy = cv2.Sobel(np.float32(img), cv2.CV_64F, 0, 1, 3)
    print("gx = ", gx)
    print("gy = ", gy)

    # Conversion of Cartesian coordinates to polar
    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    print("Magnitude = ", mag)
    print("Angle = ", ang)

    mag_max = np.max(mag)
    print("Maximum magnitude = ", mag_max)

    if not weak_th: weak_th = mag_max * 0.1
    if not strong_th: strong_th = mag_max * 0.7
    height, width = img.shape

    for i_x in range(width):
        for i_y in range(height):
            grad_ang = ang[i_y, i_x]

            grad_ang = ang[i_y, i_x]
            grad_ang = abs(grad_ang - 180) if abs(grad_ang) > 180 else abs(grad_ang)

            if grad_ang <= 45:
                mag[i_y, i_x] = 0
                continue

            # top right (diagnol-1) direction
            elif grad_ang > 45 and grad_ang <= (45 + 11.5):
                neighb_1_x, neighb_1_y = i_x - 1, i_y - 1
                neighb_2_x, neighb_2_y = i_x + 1, i_y + 1

            # In y-axis direction
            elif grad_ang > (45+11.5) and grad_ang <= (90+11.5):
                neighb_1_x, neighb_1_y = i_x, i_y - 1
                neighb_2_x, neighb_2_y = i_x, i_y + 1

            # top left (diagnol-2) direction
            elif grad_ang > (90 + 11.5) and grad_ang <= (135):
                neighb_1_x, neighb_1_y = i_x - 1, i_y + 1
                neighb_2_x, neighb_2_y = i_x + 1, i_y - 1

            # Now it restarts the cycle
            elif grad_ang > (135) and grad_ang <= ( 180):
                mag[i_y, i_x] = 0
                continue

            # Non-maximum suppression step
            if width > neighb_1_x >= 0 and height > neighb_1_y >= 0:
                if mag[i_y, i_x] < mag[neighb_1_y, neighb_1_x]:
                    mag[i_y, i_x] = 0
                    continue

            if width > neighb_2_x >= 0 and height > neighb_2_y >= 0:
                if mag[i_y, i_x] < mag[neighb_2_y, neighb_2_x]:
                    mag[i_y, i_x] = 0


    cv2.imshow("Non-maximum Suppression", mag)
    # cv2.imwrite("Non-maximum Suppression.jpg", mag)

    for i_x in range(width):
        for i_y in range(height):

            grad_mag = mag[i_y, i_x]

            if grad_mag < weak_th:
                mag[i_y, i_x] = 0
            elif grad_mag >= strong_th:
                mag[i_y, i_x] = 255


    def hysteresis(img, weak, strong=255):
        M, N = img.shape
        for i in range(1, M - 1):
            for j in range(1, N - 1):
                if (img[i, j] == weak):
                    try:
                        if ((img[i + 1, j - 1] == strong) or (img[i + 1, j] == strong) or (img[i + 1, j + 1] == strong)
                                or (img[i, j - 1] == strong) or (img[i, j + 1] == strong)
                                or (img[i - 1, j - 1] == strong) or (img[i - 1, j] == strong) or (
                                        img[i - 1, j + 1] == strong)):
                            img[i, j] = strong
                        else:
                            img[i, j] = 0
                    except IndexError as e:
                        pass
        return img

    hysteresis_img =  hysteresis(mag, weak_th)


    cv2.imshow("Double Thresholding", mag)
    # cv2.imwrite("Double Thresholding.jpg", mag)
    cv2.imshow("Hysteresis", hysteresis_img)
    # cv2.imwrite("Hysteresis.jpg", hysteresis_img)

    return hysteresis_img

In [ ]:
img = cv2.imread("Shaqi_jrvej.jpg")
img = cv2.resize(img, (400, 400), interpolation = cv2.INTER_AREA)
cv2.imwrite("input.jpg", img)
canny_img = Canny_detector(img)
cv2.imshow("original", img)
cv2. imshow("canny_output",canny_img)
cv2.imwrite("Output 2(a).jpg", canny_img)

if cv2.waitKey(0) & 0xFF == 27:
    cv2.destroyAllWindows()

gx =  [[  0.   0.   0. ...   0.   4.   0.]
 [  0.   0.   0. ...   0.   3.   0.]
 [  0.   0.   0. ...   0.   1.   0.]
 ...
 [  0.  24.  13. ...  67.  36.   0.]
 [  0.  14. -19. ...  69.  29.   0.]
 [  0.   8. -34. ...  70.  26.   0.]]
gy =  [[  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   2.   1.   0.]
 [  0.   0.   0. ...   2.   1.   0.]
 ...
 [ 16.  12.  -9. ...  19.  16.  12.]
 [  0.  -6. -27. ...  21.  19.  16.]
 [  0.   0.   0. ...   0.   0.   0.]]
Magnitude =  [[ 0.          0.          0.         ...  0.          4.
   0.        ]
 [ 0.          0.          0.         ...  2.          3.16227766
   0.        ]
 [ 0.          0.          0.         ...  2.          1.41421356
   0.        ]
 ...
 [16.         26.83281573 15.8113883  ... 69.64194139 39.39543121
  12.        ]
 [ 0.         15.23154621 33.01514804 ... 72.12489168 34.66987165
  16.        ]
 [ 0.          8.         34.         ... 70.         26.
   0.        ]]
Angle =  [[  0.           0.           0.  